In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [3]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [4]:
fea_number=6
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A2_016.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A2_019', 'Input_A2_018', 'Input_A4_016', 'Input_A6_017',
       'Input_A6_016', 'Input_A5_019'], dtype=object)

## Linear Regression Baseline

In [5]:
lr_rmse_ave_fea(skew_data,6)

train_rmse: 0.008044798708574738
test_rmse: 0.00842174415434341
test_r2: 0.5779946965784081


## Lasso

In [6]:
laso_rmse_ave(skew_data[title],y,5e-05)

test_rmse_ave: 0.008409586376601795
[0.008075465298330036, 0.00791152396369169, 0.008917094790309695, 0.008271623607208066, 0.007606711338365201, 0.007526075286801399, 0.009575456599207865, 0.009101761342332816, 0.00821787984201634, 0.008892271697754828]


test_r2_ave: 0.5770345886918602
[0.6649257948260741, 0.6633511990408596, 0.4949721011252771, 0.5881839904552834, 0.6649461746399874, 0.6717725733826536, 0.39014656858667873, 0.5121090481994248, 0.5754855726766388, 0.5444528639857233]


## ElasticNet

In [7]:
ElasticNet_rmse_ave(skew_data[title],y,[0.02],0.01)

test_rmse_ave: 0.00839153662939823
[0.00812996697614208, 0.007878619251568257, 0.008766092393325976, 0.008306545835593546, 0.007634577632503669, 0.007580113089361061, 0.009502217942602595, 0.009062295119254738, 0.008163387894411352, 0.008891550159219011]


test_r2_ave: 0.5794087765334078
[0.660387670622838, 0.6661456786234328, 0.5119315989675998, 0.5846993319781477, 0.6624868173694487, 0.6670422554959089, 0.39943991856589856, 0.5163309709842439, 0.5810967335417028, 0.5445267891848572]


## SVR

In [8]:
model=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,6,model)

train_rmse: 0.008447081811618556
test_rmse: 0.008784714647108189
test_r2: 0.5381124015587389


## XGB

In [9]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)
xgb_ave(skew_x, y,best_xgb_model)

test_rmse_ave: 0.00880764711951124
[0.008162968029984506, 0.008577762353002276, 0.009462903638912055, 0.00855081208142069, 0.007939599625002633, 0.008189740832580842, 0.009093054026034867, 0.009414726896857857, 0.009560799731481136, 0.009124103979835523]


test_r2_ave: 0.5373193386579707
[0.6576249751286939, 0.6042646958273833, 0.4312552000067853, 0.5599151457576778, 0.6349789465584614, 0.6113326421592595, 0.4500464240450244, 0.47797977488315724, 0.42540568727536754, 0.5203898949378959]


In [10]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.0084989820590604
[0.00797442126676692, 0.007880498791456329, 0.008843353869953819, 0.008677508160940803, 0.007627026495941849, 0.007462491143935448, 0.009903671805447205, 0.009198473515063388, 0.008733635701649587, 0.008688739839448636]


test_r2_ave: 0.5664666091216988
[0.6732585480204027, 0.6659863696527217, 0.5032903342940361, 0.5467771895424935, 0.6631541359506046, 0.6772952125571414, 0.3476225101843011, 0.5016856415761222, 0.5205282479249806, 0.5650679015141846]


## RandomForest

In [26]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 7,
            min_samples_split = 5,
            max_leaf_nodes = 13,
            min_samples_leaf =2,
            random_state = 42) 


In [12]:
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.008385930877978835
[0.00815275039854773, 0.007539147462421775, 0.008919630790990507, 0.008243125803958515, 0.007255123396815386, 0.007471105994667943, 0.009491631810121028, 0.009025759235235686, 0.009240035168877976, 0.00852099871815182]


test_r2_ave: 0.5776245512724698
[0.6584815440648426, 0.6942959078459898, 0.49468480279621607, 0.5910167197248034, 0.6952032612383012, 0.6765497086239718, 0.4007773046823281, 0.520223065256559, 0.46331416968691996, 0.581699028804767]


In [14]:
lasso_mod=Lasso(alpha=5e-05)
elastic_mod=ElasticNet(alpha=[0.02], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.2, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 5, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.7,
              colsample_bytree = 0.65,
              reg_alpha =  0.02,
              reg_lambda = 1.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 7,
            min_samples_split = 5,
            max_leaf_nodes = 13,
            min_samples_leaf =2,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod),('SVR', svr_mod), ('Elastic', elastic_mod), 
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.008351779655434183
[0.008128765936216927, 0.007708471380646776, 0.008962974194980093, 0.00821847506623337, 0.007350101562935055, 0.00736266209998391, 0.009590007339521913, 0.009099585672621186, 0.00843426731040736, 0.008662485990795256]


test_r2_ave: 0.5818322569160419
[0.6604880050618857, 0.6804099093809555, 0.48976188705303625, 0.5934591585025533, 0.6871707241501136, 0.685871391896, 0.388291709533539, 0.5123422695855174, 0.5528352065396396, 0.5676923074571791]


In [21]:

stack_mod = StackingRegressor(regressors=[lasso_mod, elastic_mod, random_mod, vote_mod,svr_mod], 
                           meta_regressor=xgb_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.008696127043078487
[0.00821597386905991, 0.007377033680770246, 0.009545417566501664, 0.00858165028667476, 0.00762136695462918, 0.00768389458268179, 0.010059564021557686, 0.009118866398751202, 0.00965671235149067, 0.009100790718667776]


test_r2_ave: 0.543386420657853
[0.6531641475028112, 0.7073016267064078, 0.42129335855605576, 0.5567351204281932, 0.6636538551470226, 0.65786260297599, 0.3269229148442627, 0.510273525655392, 0.41381935982949836, 0.5228376949328957]


In [23]:
ave(skew_data[title],y,0.2,stack_mod,0.5,vote_mod,0.3,random_mod)

test_rmse_ave: 0.008322859489036476
[0.00805873671345701, 0.007502960679424672, 0.008982313823508717, 0.008169177215111744, 0.007232968996703966, 0.007376188684323312, 0.009553414004640224, 0.009013227289768382, 0.008762396342534682, 0.008577211140892048]


test_r2_ave: 0.5839225974382293
[0.6663125908250815, 0.6972235329256187, 0.4875576050002084, 0.5983217347830343, 0.697061886821742, 0.6847161058768432, 0.39295108850956895, 0.5215544467233815, 0.5173651618844602, 0.5761618210323542]


In [24]:
ave(skew_data[title],y,0.1,stack_mod,0.5,vote_mod,0.4,random_mod)

test_rmse_ave: 0.008315334514831537
[0.008075816043704772, 0.007528307487890248, 0.00893376960088383, 0.008164425312799786, 0.00722391776384969, 0.0073744274791865, 0.009521182998495458, 0.009019455820208206, 0.00875797616916704, 0.008554066472129838]


test_r2_ave: 0.5848859048884332
[0.6648966873318084, 0.6951743738104155, 0.49308154853247144, 0.5987889007371315, 0.6978195971831098, 0.6848666479374841, 0.39704026374115076, 0.5208929650464054, 0.5178519674334021, 0.5784460971309526]
